<h2>Traccar 테스트 데이터 분석</h2>
<p>일부 환경에서 Traccar 가 오랜 기간 (몇 시간) 동안 데이터를 송신하지 않는 현상을 확인해야 한다.
주로 아이폰에서 일어나는 것으로 파악.</p>

<p>실험기간: 201901</p>
<p>Traccar 서버 버전: 4.2</p>
<p>서버: cssc.hanyang.ac.kr:5055</p>
<p>주기 60초</p>
<table>
    <thead>
        <tr>
            <th>사용자                        </th>
            <th>기종</th>
            <th>OS</th>
            <th>Traccar Client</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>DBLAB0101</td>
            <td>V20</td>
            <td>8.0.0/F800K20i</td>
            <td>5.17</td>            
        </tr>
        <tr>
            <td>DBLAB0102</td>
            <td>S5</td>
            <td>6.0.1/F800K20i</td>
            <td>5.17</td>            
        </tr>
        <tr>
            <td>DBLAB0103</td>
            <td>아이폰6</td>
            <td>12.1.2/16C104</td>
            <td>5.5</td>            
        </tr>
        <tr>
            <td>DBLAB0201</td>
            <td>아이폰 XS</td>
            <td>12.1.2</td>
            <td>5.5</td>            
        </tr>
        <tr>
            <td>DBLAB0301</td>
            <td>아이폰 X</td>
            <td>12.1.3 beta 4</td>
            <td>5.5</td>            
        </tr>
        <tr>
            <td>HCICG01</td>
            <td>?</td>
            <td>?</td>
            <td>?</td>            
        </tr>
    </tbody>
</table>

<p>어쩌다 알아내게 된 것: Traccar 클라이언트가 신호를 올바르게 보내지만 서버에 등록되어있지 않았던 경우 다시 등록하면 한방에 다 보낸다...</p>
<p>HCICG0160 실험자 자료 (1월 9일 종료)</p>

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [2]:
dbconfig = json.loads(open("res/dbconfig.properties").read())
engine = create_engine(URL(**dbconfig["cssc_jaunt"]))

In [3]:
sql = "select * from tc_devices;"
tester = pd.read_sql_query(sql, engine)

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "cssc.hanyang.ac.kr" (166.104.142.233) and accepting
	TCP/IP connections on port 5432?
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
tester

In [ ]:
pos_query = "select d.name, p.* from tc_positions p, tc_devices d where p.deviceid = d.id and d.id IN (select id from tc_devices where name=ANY({$users}))"
users = tester['name'].to_json(force_ascii=False, orient='values')
#json.dumps()
users

In [76]:
users = list(users)
users[0] = '{'
users[-1] = '}'
users = "".join(users)
users

'{"HCICG0160","이상진","정s","박민우","정i","정근성"}'

In [77]:
pos_query = "select d.name, p.* from tc_positions p, tc_devices d where p.deviceid = d.id and d.id IN (select id from tc_devices where name=ANY('{0}'))".format(users)
pos_query

'select d.name, p.* from tc_positions p, tc_devices d where p.deviceid = d.id and d.id IN (select id from tc_devices where name=ANY(\'{"HCICG0160","이상진","정s","박민우","정i","정근성"}\'))'

In [78]:
#pos_query += "LIMIT 10"
df = pd.read_sql_query(pos_query, engine)

In [79]:
df

,name,id,protocol,deviceid,servertime,devicetime,fixtime,valid,latitude,longitude,altitude,speed,course,address,attributes,accuracy,network
0,이상진,286386,osmand,48,2019-01-16 14:49:08.835,2019-01-16 14:49:06,2019-01-16 14:49:06,True,37.555166,127.050135,26.661201,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.000,null
1,이상진,286387,osmand,48,2019-01-16 14:50:05.620,2019-01-16 14:49:40,2019-01-16 14:49:40,True,37.555168,127.050031,27.668663,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":9.18,""totalDis...",0.000,null
2,이상진,286388,osmand,48,2019-01-16 14:50:22.141,2019-01-16 14:50:05,2019-01-16 14:50:05,True,37.555143,127.050146,27.339186,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":10.52,""totalDi...",0.000,null
3,이상진,286389,osmand,48,2019-01-16 14:50:31.135,2019-01-16 14:50:28,2019-01-16 14:50:28,True,37.555248,127.050002,27.484837,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":17.27,""totalDi...",0.000,null
4,이상진,286390,osmand,48,2019-01-16 14:50:35.049,2019-01-16 14:50:35,2019-01-16 14:50:35,True,37.555233,127.049978,27.477942,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":2.7,""totalDist...",0.000,null
5,이상진,286391,osmand,48,2019-01-16 14:50:53.960,2019-01-16 14:50:35,2019-01-16 14:50:35,True,37.555233,127.049978,27.477942,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.000,null
6,이상진,286392,osmand,48,2019-01-16 14:50:54.008,2019-01-16 14:50:44,2019-01-16 14:50:44,True,37.555233,127.049978,27.477942,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.000,null
7,이상진,286393,osmand,48,2019-01-16 14:50:54.034,2019-01-16 14:50:54,2019-01-16 14:50:54,True,37.555188,127.049901,27.477942,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":8.44,""totalDis...",0.000,null
8,이상진,286394,osmand,48,2019-01-16 14:51:00.751,2019-01-16 14:51:00,2019-01-16 14:51:00,True,37.555247,127.049911,28.168867,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":6.63,""totalDis...",0.000,null
9,이상진,286395,osmand,48,2019-01-16 14:51:07.155,2019-01-16 14:51:07,2019-01-16 14:51:07,True,37.555221,127.050030,27.502682,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":10.89,""totalDi...",0.000,null


In [80]:
json.loads(df['attributes'][0])

{'batteryLevel': 74.0, 'distance': 0.0, 'totalDistance': 0.0, 'motion': False}

In [81]:
df["battery_level"] = df['attributes'].map(lambda x: json.loads(x)['batteryLevel'])
df["dist"] = df['attributes'].map(lambda x: json.loads(x)['distance'])
df["total_dist"] = df['attributes'].map(lambda x: json.loads(x)['totalDistance'])
df["motion"] = df['attributes'].map(lambda x: json.loads(x)['motion'])

In [82]:
df

,name,id,protocol,deviceid,servertime,devicetime,fixtime,valid,latitude,longitude,...,speed,course,address,attributes,accuracy,network,battery_level,dist,total_dist,motion
0,이상진,286386,osmand,48,2019-01-16 14:49:08.835,2019-01-16 14:49:06,2019-01-16 14:49:06,True,37.555166,127.050135,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.000,null,74.0,0.00,0.00,False
1,이상진,286387,osmand,48,2019-01-16 14:50:05.620,2019-01-16 14:49:40,2019-01-16 14:49:40,True,37.555168,127.050031,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":9.18,""totalDis...",0.000,null,74.0,9.18,9.18,False
2,이상진,286388,osmand,48,2019-01-16 14:50:22.141,2019-01-16 14:50:05,2019-01-16 14:50:05,True,37.555143,127.050146,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":10.52,""totalDi...",0.000,null,74.0,10.52,19.70,False
3,이상진,286389,osmand,48,2019-01-16 14:50:31.135,2019-01-16 14:50:28,2019-01-16 14:50:28,True,37.555248,127.050002,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":17.27,""totalDi...",0.000,null,74.0,17.27,36.97,False
4,이상진,286390,osmand,48,2019-01-16 14:50:35.049,2019-01-16 14:50:35,2019-01-16 14:50:35,True,37.555233,127.049978,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":2.7,""totalDist...",0.000,null,74.0,2.70,39.67,False
5,이상진,286391,osmand,48,2019-01-16 14:50:53.960,2019-01-16 14:50:35,2019-01-16 14:50:35,True,37.555233,127.049978,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.000,null,74.0,0.00,39.67,False
6,이상진,286392,osmand,48,2019-01-16 14:50:54.008,2019-01-16 14:50:44,2019-01-16 14:50:44,True,37.555233,127.049978,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.000,null,74.0,0.00,39.67,False
7,이상진,286393,osmand,48,2019-01-16 14:50:54.034,2019-01-16 14:50:54,2019-01-16 14:50:54,True,37.555188,127.049901,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":8.44,""totalDis...",0.000,null,74.0,8.44,48.11,False
8,이상진,286394,osmand,48,2019-01-16 14:51:00.751,2019-01-16 14:51:00,2019-01-16 14:51:00,True,37.555247,127.049911,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":6.63,""totalDis...",0.000,null,74.0,6.63,54.74,False
9,이상진,286395,osmand,48,2019-01-16 14:51:07.155,2019-01-16 14:51:07,2019-01-16 14:51:07,True,37.555221,127.050030,...,0.000000,0.000000,None,"{""batteryLevel"":74.0,""distance"":10.89,""totalDi...",0.000,null,74.0,10.89,65.63,False


In [83]:
# attributes are flatted
# addresses are N/A in Korea
df.drop(["attributes", "address"], axis=1, inplace=True)

In [86]:
df.shape

(23734, 20)

In [94]:
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df.drop(cols_to_drop, axis=1, inplace=True)

In [102]:
df

,name,id,deviceid,servertime,devicetime,fixtime,latitude,longitude,altitude,speed,course,accuracy,battery_level,dist,total_dist,motion
0,이상진,286386,48,2019-01-16 14:49:08.835,2019-01-16 14:49:06,2019-01-16 14:49:06,37.555166,127.050135,26.661201,0.000000,0.000000,0.000,74.0,0.00,0.00,False
1,이상진,286387,48,2019-01-16 14:50:05.620,2019-01-16 14:49:40,2019-01-16 14:49:40,37.555168,127.050031,27.668663,0.000000,0.000000,0.000,74.0,9.18,9.18,False
2,이상진,286388,48,2019-01-16 14:50:22.141,2019-01-16 14:50:05,2019-01-16 14:50:05,37.555143,127.050146,27.339186,0.000000,0.000000,0.000,74.0,10.52,19.70,False
3,이상진,286389,48,2019-01-16 14:50:31.135,2019-01-16 14:50:28,2019-01-16 14:50:28,37.555248,127.050002,27.484837,0.000000,0.000000,0.000,74.0,17.27,36.97,False
4,이상진,286390,48,2019-01-16 14:50:35.049,2019-01-16 14:50:35,2019-01-16 14:50:35,37.555233,127.049978,27.477942,0.000000,0.000000,0.000,74.0,2.70,39.67,False
5,이상진,286391,48,2019-01-16 14:50:53.960,2019-01-16 14:50:35,2019-01-16 14:50:35,37.555233,127.049978,27.477942,0.000000,0.000000,0.000,74.0,0.00,39.67,False
6,이상진,286392,48,2019-01-16 14:50:54.008,2019-01-16 14:50:44,2019-01-16 14:50:44,37.555233,127.049978,27.477942,0.000000,0.000000,0.000,74.0,0.00,39.67,False
7,이상진,286393,48,2019-01-16 14:50:54.034,2019-01-16 14:50:54,2019-01-16 14:50:54,37.555188,127.049901,27.477942,0.000000,0.000000,0.000,74.0,8.44,48.11,False
8,이상진,286394,48,2019-01-16 14:51:00.751,2019-01-16 14:51:00,2019-01-16 14:51:00,37.555247,127.049911,28.168867,0.000000,0.000000,0.000,74.0,6.63,54.74,False
9,이상진,286395,48,2019-01-16 14:51:07.155,2019-01-16 14:51:07,2019-01-16 14:51:07,37.555221,127.050030,27.502682,0.000000,0.000000,0.000,74.0,10.89,65.63,False


In [109]:
#Daily data by users needed
#Get tested days

df['devicetime'].min(), df['devicetime'].max()

(Timestamp('2019-01-16 14:49:06'), Timestamp('2019-01-24 17:51:15'))

In [ ]:

for user in tester["name"]:
    print(user)
    print(df[df["name"] == user].count())
    